In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 디렉토리 지정
cd /content/drive/My Drive/Colab/data

/content/drive/My Drive/Colab/data


# **콘텐츠 기반 필터링 영화 추천 시스템**

In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np

movies = pd.read_csv('tmdb_5000_movies.csv', encoding='utf-8')
movies[:1]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
# 1. 데이터 전처리

# movies에서 유사도를 비교할 컬럼 8개만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count','popularity', 'keywords', 'overview']].copy()

# literal_eval : dictionary 형태로 저장된 genres의 string 값을 실제 dictionary로 가공
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
# keywords도 동일하게 수행
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])

# 리스트에 담겨있는 genres, keywords를 공백 기준으로 문자열로 붙이기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))
movies_df[:1]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,keywords_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...


In [ ]:
# 2. 텍스트 데이터를 Feature Vectorization 
# (Countvectorizer를 사용하였으며, 경우에 따라 Lemmatization도 추가적으로 사용할 수 있음)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CountVectorizer : 문서를 토큰화하고, 토큰의 출현빈도를 바탕으로 각 문서를 BOW 인코딩 벡터로 변환
# min_df : 단어장에 포함되기 위한 최소 빈도
cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))

# fit_transform 안에 데이터프레임 형태로 넣어주면 안 됨. 변수 하나씩만 넣어주자!
genres_vect = cnt_vect.fit_transform(movies_df['genres_literal'])
keywords_vect = cnt_vect.fit_transform(movies_df['keywords_literal'])

# 장르에 따른 영화별 코사인 유사도 추출
genre_sim = cosine_similarity(genres_vect, genres_vect)

# 유사도행렬값 3개만 출력
print(genre_sim[:3])

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [ ]:
# 3. 유사도를 기준으로 'The Avengers' 영화와 유사한 영화 10개 확인

# argsort : 내림차순으로 정렬 후 순서대로 데이터의 위치를 반환 (유사도가 높은 영화들의 index 추출)
genre_sim_idx = genre_sim.argsort()[::-1]
print(genre_sim_idx)
print("\n")

# 특정 영화를 기준으로 유사도가 높은 영화 10개를 반환하는 함수
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 비교기준으로 선정할 특정 영화 선정
    title_movie = df[df['title'] == title_name]
    
    # 비교기준 영화의 index.values를 할당해 유사도 행렬에서 비교기준 영화에 해당하는 유사도 행렬값을 찾자!
    title_movie_idx = title_movie.index.values
    # 모든 영화끼리의 유사도 행렬에서 비교기준 영화에 대한 유사도행렬 찾아서 할당
    top_sim_idx = sorted_idx[title_movie_idx, :top_n]
    print(top_sim_idx)
    print("\n")
    
    # top_sim_idx는 2차원 array이기 때문에 1차원 array로 변경
    # 왜냐하면 top_sim_idx로 movies_df의 index값으로 넣어서 유사한 영화를 추출할 것이기 때문
    top_sim_idx = top_sim_idx.reshape(-1,)
    similar_movie = df.iloc[top_sim_idx]
    
    return similar_movie
    
similar_movies = find_sim_movie(movies_df, genre_sim_idx, 'The Avengers')
print(similar_movies[['title','vote_average','vote_count']])

[[   0 3141 3140 ... 4521 4710 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [   0 2230 2229 ... 1895 3809 4800]
 ...
 [2401 2999 3000 ... 1542 1740    2]
 [2401 3067 3069 ...  129    1  262]
 [2401 3037 3038 ...  813 3494    0]]


[[   0 2743 2749 2750 2751 2752 2755 2757 2760 2764]]


                      title  vote_average  vote_count
0                    Avatar           7.2       11800
2743   The Butterfly Effect           7.3        2060
2749         Child's Play 2           5.8         308
2750           No Good Deed           5.6         181
2751               The Mist           6.7        1399
2752             Ex Machina           7.6        4737
2755          Earth to Echo           5.7         290
2757  Letters from Iwo Jima           7.2         541
2760                   Room           8.1        2757
2764            Light It Up           6.6           7


In [ ]:
# 4. 유사도와 가중 평점을 기준으로 'The Avengers' 영화와 유사한 영화 10개 확인

C = movies_df['vote_average'].mean()

# quantile : 분위수. 투표횟수 값들의 중앙값(50%)
m = movies_df['vote_count'].quantile(0.5)
print("\n",m)

# 투표횟수와 평점을 사용해 가중 평점을 계산하는 함수
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)
    
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
 
# 가중 평점 변수를 추가해서 유사한 영화 찾아주는 함수 재정의
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_idx = title_movie.index.values
    
    similar_idx = sorted_idx[title_idx, :(top_n*2)]
    similar_idx = similar_idx.reshape(-1,)
    
    #자기 자신 영화 제외, boolean index기법 사용!
    similar_idx = similar_idx[similar_idx != title_idx]
    return df.iloc[similar_idx].sort_values(by=['weighted_vote'], ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_idx, 'The Avengers')
print(similar_movies[['title','vote_average','weighted_vote']])


 235.0
                      title  vote_average  weighted_vote
2760                   Room           8.1       7.942300
2752             Ex Machina           7.6       7.528733
0                    Avatar           7.2       7.178368
2743   The Butterfly Effect           7.3       7.176323
2757  Letters from Iwo Jima           7.2       6.864511
2751               The Mist           6.7       6.612583
2783               The Call           6.6       6.520014
2787         Quest for Fire           7.1       6.411513
2742           The Shallows           6.2       6.185938
2778               Infamous           6.4       6.154781
